In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
from fastai.text import *

/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may

In [3]:
from model.data_utils import minibatches, pad_sequences, get_chunks
from model.config import Config
from model.data_utils import CoNLLDataset

In [4]:
config = Config()

In [5]:
train = CoNLLDataset(config.filename_train, config.processing_word,
                         config.processing_tag, config.max_iter)
val = CoNLLDataset(config.filename_dev, config.processing_word,
                         config.processing_tag, config.max_iter)
test = CoNLLDataset(config.filename_test, config.processing_word,
                         config.processing_tag, config.max_iter)

In [6]:
class minibatches(data, minibatch_size):
    def __init__(self, data, minibatch_size):
        self.data = data
        self.minibatch_size = minibatch_size
        
    def 
    x_batch, y_batch = [], []
    for (x, y) in data:
        if len(x_batch) == minibatch_size:
            char_ids, word_ids = zip(*x_batch)
            word_ids, sequence_lengths = pad_sequences(word_ids, 0)
            char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
                nlevels=2)
            labels, labels_idx = pad_sequences(y_batch, 0)
            yield [T(char_ids), T(word_ids)], [T(labels)]
            x_batch, y_batch = [], []

        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]

    if len(x_batch) != 0:
        yield x_batch, y_batch

In [7]:
class NER_model(nn.Module):
    def __init__(self, config):
        super(NER_model, self).__init__()
        self.config = config
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}
        self.get_word_embeddings()
        self.get_logits()
        
    def get_word_embeddings(self):
        # get word embeding
        _word_embedding = V(self.config.embeddings, requires_grad=True)
        self.word_embedding = nn.Embedding.from_pretrained(_word_embedding)
        
        # get char embedding
        self._char_embedding = nn.Embedding(self.config.nchars, self.config.dim_char)
        self.char_embedding = nn.LSTM(input_size=self.config.dim_char, hidden_size=self.config.hidden_size_char,
                                     num_layers=1, batch_first=True, # not sure here whether batch is first
                                     bidirectional=True)
        
    def get_logits(self):
        self.rnn = nn.LSTM(input_size=self.config.dim_word+self.config.dim_char*2,
                          hidden_size=self.config.hidden_size_lstm,
                          num_layers=1, batch_first=True, # not sure whether batch is first
                          bidirectional=True)
        self.dropout = nn.Dropout(self.config.dropout if self.train else 0)
        self.linear = nn.Linear(self.config.hidden_size_lstm*2, self.config.ntags)
        
    def forward(self, input):
        char = input[0]
        words = input[1]
        bs, sl, _ = char.size()
        char = char.view(-1, _)
        _char_embedding = self._char_embedding(char)
        
        char_embedding, (h_n, cell_n) = self.char_embedding(_char_embedding)
        char_embedding = h_n.view(bs, sl, -1)
        word_embedding = self.word_embedding(words)

        # concat word embeddings and char embeddings
        word_embedding = torch.cat([word_embedding, char_embedding], dim=-1)
        word_embedding_dp = self.dropout(word_embedding)
        
        out, (n_h, n_cell) = self.rnn(word_embedding_dp)
        out_dp = self.dropout(out)
        out = self.linear(out_dp)
        
        return out

In [8]:
# for i, (data, label) in enumerate(minibatches(train, 100)):
#     char_ids, word_ids = zip(*data)
#     word_ids, sequence_lengths = pad_sequences(word_ids, 0)
#     char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
#         nlevels=2)

In [84]:
trn_dl = minibatches(train, 20)
val_dl = minibatches(val, 20)
test_dl = minibatches(test, 20)
dir_path = '/home/'
md = ModelData(dir_path, trn_dl, val_dl, test_dl)

In [10]:
ner_model = NER_model(config).cuda()

In [59]:
# def fbeta_torch(y_true, y_pred, beta, threshold, eps=1e-9):
#     y_pred = (y_pred.float() > threshold).float()
#     y_true = y_true.float()
#     tp = (y_pred * y_true).sum(dim=1)
#     precision = tp / (y_pred.sum(dim=1)+eps)
#     recall = tp / (y_true.sum(dim=1)+eps)
#     return torch.mean(
#         precision*recall / (precision*(beta**2)+recall+eps) * (1+beta**2))

In [65]:
opt = optim.Adam(ner_model.parameters(), 1e-3)
loss_func = F.nll_loss

In [85]:
fit(ner_model, md, 4, opt, F.nll_loss)

TypeError: object of type 'generator' has no len()

In [82]:
epoch = 4
for i in range(epoch):
    for step, (X_batch, Y_batch) in enumerate(trn_dl):
        print(step)
        Y = ner_model(X_batch)
        
#         bs, sl = Y_batch[0].size()
#         Y_batch = Y_batch[0].view(-1)
        

#         Y_onehot = torch.zeros(bs*sl, 8).cuda()
#         print(Y_onehot.size())
#         Y_onehot.scatter_(1, Y_batch)
        _, prediction = torch.max(Y, dim=-1)
        print(prediction.size())
        print(Y_batch[0].size())
        print(V(prediction))
        loss = loss_func(prediction, Y_batch[0])
        opt.zero_grad()
        loss.backward()
        opt.step()

0
torch.Size([20, 13])
torch.Size([20, 13])
tensor([[0, 4, 3, 1, 5, 1, 5, 1, 1, 1, 2, 1, 4],
        [3, 3, 1, 1, 3, 7, 7, 3, 3, 3, 5, 3, 5],
        [1, 1, 1, 3, 3, 3, 3, 1, 3, 3, 3, 6, 1],
        [3, 7, 1, 3, 1, 7, 1, 0, 4, 3, 3, 5, 1],
        [1, 1, 2, 1, 6, 1, 7, 3, 6, 1, 4, 3, 3],
        [1, 3, 3, 3, 5, 5, 1, 5, 3, 3, 7, 0, 4],
        [3, 1, 3, 1, 3, 3, 3, 3, 6, 3, 1, 0, 2],
        [7, 0, 1, 3, 5, 5, 5, 5, 6, 4, 1, 0, 3],
        [3, 0, 3, 0, 3, 6, 3, 6, 3, 3, 1, 5, 3],
        [6, 1, 3, 6, 5, 5, 5, 3, 3, 3, 3, 3, 0],
        [4, 4, 5, 5, 5, 5, 5, 4, 4, 7, 5, 4, 7],
        [1, 0, 0, 5, 5, 0, 0, 0, 6, 0, 3, 5, 5],
        [3, 7, 6, 3, 1, 6, 6, 5, 4, 0, 5, 3, 5],
        [5, 1, 6, 2, 2, 2, 4, 4, 4, 6, 4, 7, 1],
        [1, 3, 3, 3, 3, 7, 7, 4, 1, 0, 3, 7, 1],
        [1, 3, 7, 1, 3, 1, 4, 4, 5, 5, 1, 7, 6],
        [4, 3, 5, 4, 7, 7, 7, 3, 3, 0, 0, 0, 3],
        [4, 7, 4, 7, 7, 4, 3, 1, 3, 3, 7, 6, 4],
        [5, 5, 5, 5, 4, 3, 5, 3, 6, 3, 4, 7, 0],
        [1, 4, 0, 4, 6, 1

ValueError: Expected 2 or more dimensions (got 1)